# Documentation -- Milestone 2 

## Introduction

Differentiation is great. It is a necessity in a vast range of applications, such as atomic simulations, economic analysis, and machine learning. 

In the field of computational science, there are three ways to differentiate: numerical, symbolic and automatic. 

Numerical differentiation finds the derivative using finite difference approximations $\Delta f / \Delta x$. Even with higher-order methods, its error is far greater than machine precision.

Symbolic differentiation finds the symbolic expression of the derivative. When functions and programs get complicated, it becomes inefficient and messy. This is called expression swell.

Automatic differentiation can find the derivative of expressions to the accuracy of machine precision. It does not have the problem of expression swell because it deals with numbers. That is why we need automatic differentiation!

Our `superjacob` package performs automatic differentiation on single- or multi-variable functions using the _forward mode_ as well as the _reverse mode_. The function is stored as an `Expression` object that can output values and derivatives at any given point. The package is named after Karl Jacob Jacobi, the mathematician who invented the Jacobian matrix. 


## Background

Differentiation is the process of finding derivative, which is the rate of change of a function's output with regard to its variables. Take $f(x,y) =3*x^2+\exp(y)$ as an example. Symbolic differentiation gives $\dfrac{\partial f}{\partial x}=6x$ and $\dfrac{\partial f}{\partial y}=\exp(y)$.

Automatic differentiation treats a function as a chain of elementary functions and performs differentiation on each elementry function. 
Here the elementary functions include: (1) A single arithmetic operation, such as $3*x$ and $x_1+x_2$. (2) A single trigonometric operation, such as $\sin(x)$. (3) A single exponential or logarithmic opration, such as $\log(x)$.

The chain rule dictates that 

$$\frac{df(g(x))}{dx}=\frac{df(x_1)}{dx_1}*\frac{dg(x)}{dx}.$$

Therefore, a function that is made up of elementary functions can be extended into a computational graph. For $f(x,y) =3*x^2+\exp(y)$, the graph is shown below. Each $x_i$ is the output of an elementary function.

<img src="https://i.imgur.com/hBQvv4n.jpg" alt="drawing" width="600"/>
  
To calulate the derivative of $f$ at $[x,\ y]$, we pass the value of the previous $x_i$ and $x_i^\prime$ into the next elementary function to evaluate the derivative of that elementary function. Below shows the forward mode automatic differentiation table (traceplot). 

<img src="https://i.imgur.com/1AIngxT.png" alt="drawing" width="600"/>

The derivative is computed using the chain rule. To get $\dfrac{\partial f}{\partial x}$, forward mode starts from $\dfrac{\partial x_1}{\partial x}$, while the reverse mode starts from $\dfrac{\partial x_6}{\partial x_4}$. 

Let's do the forward mode at a given point x=2, y=3. Start from the begining, plug in numbers for each step, we have

$$\dfrac{\partial f}{\partial x} = \dfrac{\partial x_6}{\partial x_4}\dfrac{\partial x_4}{\partial x_3}\dfrac{\partial x_3}{\partial x_1}\dfrac{\partial x_1}{\partial x}=1*3*4*1=12.$$

## How to use superjacob

Our goal is for the syntax of `superjacob` to be as natural as possible, not requiring the user to learn any new paradigms and thereby minimizing the chances of errors. Therefore, we take inspiration from the kind of notation one might use when writing out mathematical expressions and functions by hand. 

The core functionality of `superjacob` involves three main kinds of objects: `Variable`, `Expression`, and subclasses of `Operation`. These mean exactly what you might expect from a mathematical context. If a user wants to define an expression, they first define one or more `Variable`s. The they make an `Expression` using basic math operators such as `+ - * / ` or special operators such as `superjacob.log`. The expression can be evaluated and differntiated at any given point.


### 1. How to install `superjacob`

There are two ways to install `superjacob`:

1\. PyPi

In the commmand window, type 

```python
pip install superjacob
```

2\. GitHub 

First, clone our GitHub repository (https://github.com/ADMonsters/cs207-FinalProject.git).

(Optional) Activate a virtual enviromment using `conda` or `virtualenv`.

In the command window, cd to our repositiory folder. Then type 

```python
pip install -r requirements.txt

pip install .
```

### 2. Demo for single variable function
```python
import superjacob as sj

# Define a variable
x = sj.Var('x')

# A variable behaves like the identity operation
x(-24)
>>> -24

# It has a derivative of 1
x.deriv(-24)
>>> 1

# You can easily create an expression object
f = (x * 0.2 + sj.exp(x) / 3) / x

# We can evaluate it just like a function
f(2)
>>> 1.431509349821775

# As well as take the derivative using forward mode
f.deriv(2)
>>> 0.6157546749108876

# To define more complex functions, we can combine previously-made expressions
g = f + sj.log(x)
```

### 3. Demo for multi-variable function and vector function

```python
import superjacob as sj

# Define multiple variables
x = sj.Var('x')
y = sj.Var('y')

# Use make_expression to make a expression
f = sj.make_expression(x*y + x/y, vars = [x, y])

# Values and derivatives
f(2, 4)
>>> 8.5

f.deriv(2, 4)
>>> array([4.25 , 1.875])

# You can specify the order of variables. Here we swap the order of x and y
g = sj.make_expression(x*y + x/y, vars = [y, x])
f(2, 4) == g(4, 2)
>>> True

# To make a vector function, separate scalar functions by a comma
fv = sj.make_expression(x*y, x/y, sj.log(x,y), vars = [x, y])

# Vector function derivative is a Jacobian
fv(2,4)
>>> [8, 0.5, 0.5]

fv.deriv(2,4)
>>> array([[ 4.      ,    2.   ],
        [ 0.25     ,  -0.125  ],
        [ 0.36067376, -0.09016844]])

```


### 4. Demo for `__str__` and `__eq__` methods

```python
import superjacob as sj

x = sj.Var('x')

f = x**3 + sj.log(x,2) - sj.cos(x)

# You can print the expression
print(f)
>>> x^3 + log_2(x) - cos(x)

# Two expressions are equal if their strings are equal. But they are not equal if the commutative law is applied
g = x**3 + sj.log(x,2) - sj.cos(x)
gc = x**3 - sj.cos(x) + sj.log(x,2)

f == g
>>> True

f == gc
>>> False

```

### 5. List of all math functions 

```python

# Addition, subtraction, multiplication, division, and power are binary operations
f = x + 10
f = x - 10
f = x * 10
f = x / 10
f = x ** 10

# Negation is a unary operation
f = - x

# Square root is a unary operation
f = sj.sqrt(x)

# The base in log can be defined. Default base = e
f = sj.log(x)
f = sj.log(x, 10)

# Trigonometric functions are unary operations
f = sj.sin(x)
f = sj.cos(x)
f = sj.tan(x)
f = sj.sec(x)
f = sj.csc(x)
f = sj.cot(x)
f = sj.arcsin(x)
f = sj.arccos(x)
f = sj.arctan(x)

# Hyperbolic functions are unary operations
f = sj.sinh(x)
f = sj.cosh(x)
f = sj.tanh(x)

# Logistic function. Default is k=1, x0=0, L=1
f = sj.logistic(x)
f = sj.logistic(x, k=2, x0=2, L=2)

# Currently superjacob does not support complex numbers
```

### 6. Demo for reverse mode

```python

# Choose forward and reverse in the deriv method. Default is 
# forward mode for single variable functions and reverse for multivariable functions
f= x**3
f.deriv(2, mode = 'forward')
f.deriv(2, mode = 'reverse')

```

## Software Organization
### 1. Directory structure
```
cs207-FinalProject/
|
|-- superjacob/
|   |-- __init__.py
|   |-- superjacob.py
|   |-- expression.py
|   |-- operations.py
|   |-- reverse.py
|
|-- tests/
|   |-- test_operations.py
|   |-- test_expression.py
|   |-- test_expression_multivar.py
|
|-- docs/
|   |-- milestone1.md
|   |-- milestone2.ipynb
|   |-- documentation.ipynb
|
|-- README.md
|-- requirements.txt
|-- setup.py
|-- LICENSE
```

`superjacob` subdirectory hosts our code. It is set up as a package through `__init__.py`.

`tests/` subdirectory hosts tests to the code.

`docs/` subdirectory hosts our documents. The documents provides an introduction to automatic differntiation, as well as a guide to using our package.

### 2. Basic modules and their functionality

Our modules are `superjacob.py`, `expression.py`, and `operations.py`. 

1. `superjacob.py` imports everthing from `expression.py` and `operations.py`. This contains the user's primary interface through the `make_expression` function, which abstracts away the graph structure of autodiff and easily allow the user to create an expression. It also contains a set of basic math operations the user can utilize in their expressions, like $sin(x)$ or $exp(x)$. 

2. `expression.py` contains the code for our `Variable` and `Expression` classes. These objects build the computational graph in a tree-like structure. Users can call the `eval()` and `deriv()` methods of an expression to get values and derivatives.

3. `operations.py` contains elementary function classes. Each elementary function have methods to build new expressions, to evaluate at given points, and to compute the derivative at given points.

4. `reverse.py` contains classes to perform the reverse mode differentiation.

### 3. Where do tests live? How are they run? How are they integrated?

As shown in the directory structure, our test suite lives inside the `tests/` subdirectory. 

The module names indicate what they are testing. In each module, there are many unit tests to ensure the differentiator run correctly and handle edge cases, including type checking, appropriately for a variety of complex functions. They also ensure that basic math operators are properly overloaded when users create expressions and evaluate them at certain points.

- `test_operations.py` more closely tests many of the math operations abstracted away under the hood to ensure quality is maintained beneath all the layers.

- `test_expression.py` and `test_expression_multivar.py` more closely ensures that `Expression` objects are instantiated properly and return correct evaluations and deriatives. 

`pytest` runs the test suite. TravisCI performed integration testing as we built the package to ensure proper functionality among all the moving pieces in our code, and it helped flag defects as they arose and maintained quality control among the various components in the software. In addition, CodeCov helped us analyze ways to improve our test suite to maintain high coverage of our code.

### 4. How to install our package

How to install is in the previous section. To develop this package, a user should activate a virtual environment. Then in the repository folder, type
```python
pip install -r requirements.txt
pip install .
```


## Implementation
### 1. Core data structures

The function to be differentiated (henceforth referred to as ƒ) will be parsed and converted into a directed graph, containing node-like objects for each step in the traceplot (i.e. each node represents an elementary operation in ƒ). The edges of the graph represent steps from one part of the traceplot to the next. This is similar to the tree structure that we learned in class, but it is built from the leaf nodes to the root.

Each node is an `Expression` object that contains
- The type of elementary operation being performed. It could be add, mul, pow, log, etc. 
- References to the mathematical objects ('parents') that go into this operation. A binary operation has two parents, while a unary operation has only one parent. The parent could be an `Expression` object, a `Variable` object, or just a number.
- Notice that a node does not reference the next operation to be performed. 

Say we have the following situation:

![](https://i.imgur.com/p2gMe9B.png)


In this case, C knows about A and B, but not about F. This may seem counterintuitive, since in the forward mode of autodiff, we need to go from A to C to F. However, we want to allow for situations like the following:

![](https://i.imgur.com/eWljQhb.png)


Here, the function `f3` is composed of two inputs (`f1` and `f2`) and combines them in an operation in node G. Rather than copying functions `f1` and `f2` into brand new graphs, we believe that it would be more memory- and time-efficient to simply refer to the same graph objects that `f1` and `f2` refer to. However, this creates a potential issue if we were to implement the graphs as bidirectional, rather than unidirectional: if we add a connection from C to G, then if the user tries to run the forward mode of autodiff on `f1`, the algorithm will continue past node C onto node G. However, node G is not part of function `f1`!

This design choice has a tradeoff, namely that each time forward mode auto differentiation is performed, Python must step from the end of the function all the way to the beginning leaf nodes. This is done by recursively calling the `eval()` and `deriv()` methods.

### 2. Core classes

<img src="https://i.imgur.com/ST3mu2D.png" alt="drawing" width="800"/>


`Var` class is the variable in the eventual function. It overloads math operators such as `__add__` and `__pow__`.


`Expression` class inherits from `Var`. It stores the user-defined functions. As shown in the demo, `f` and `g` are expressions.


`BaseOperation` and its subclasses. Each operation has class methods to return new expression objects, to return the numerical value, and to return the drivative.


### 3. Important attributes

`Var` class

- `eval()` method. Var object do not store values. It returns whatever value that is being passe into it.
- `deriv()` method. If we are differentiating with respect to this particular variable, it returns 1. If we are differentiating with respect to another variable, it returns 0. This is done by passing a boolean as the input.
- Math operators such as `__add__()`. It calls `superjacob.add`, which returns an Expression object.


`Expression` class

- All methods defined in `Variable`
- `self.parents`. The parents can be an Expression object, a Var object, or a number.
- `self.operation` stores the operation that is being performed.
- `self._vars` stores the list of variables but do not store their values. 
- `eval()` and `deriv()` use helper methods to recursively find the value and derivative at any given point. Currently `deriv` only works with the forward mode.
- `_parse_args()` and `_get_input_args()` are helper methods that take in numerical input and smartly passes the values to the parent expressions. They can handle the situation where two parent expressions use different variables, e.g. `f(x) * g(y)`.


Operation classes such as `Add` and `Log`

- `expr()` takes in parent nodes and returns a new Expression object.
- `eval()` takes in numbers to evaluate this operation at any given point.
- `deriv()` takes in numbers to find the derivative at any given point. For example, `Mul.deriv()` returns `num1 * deriv2 + num2 * deriv1`.

`make_expression` function 

- It returns an expression with the varlist in the specified order. In this way, a user can control the order of the variable list.

```python
x = sj.Var('x')
y = sj.Var('y')
f1 = sj.make_expression(sd.log(x)/ y, vars = [x, y])
f2 = sj.make_expression(sd.log(x)/ y, vars = [y, x])

```


### 4. External dependencies

* `numpy`
* `pytest` (only for the test suite, not for the actual functionality)

### 5. Elementary functions

The elementary functions are coded in module `operations.py`. As described above, each operation has its own class. All the operations are listed in the how to use section.


## Extension Feature - Reverse Mode

In many cases, the function to be differentiated is a multivariate function with many inputs and few outputs. This is a common situation in machine learning, where loss functions take as parameters all the weights (in the case of a neural net) or coefficients (in the case of a regression) and output a scalar loss value. The major limitation of forward mode is that it requires a pass through the computational graph for each input, quickly rendering functions of this kind too costly to differentiate.

**Reverse mode** of automatic differentiation avoids this flaw by breaking up the differentiation process into two components: a *forward pass* and a *reverse pass*. As will be explained subsequently, the result is an algorithm that is able to compute the derivative of the function with respect to each input variable in a single loop, resulting in significant computational gains.

It is for this reason that **reverse mode** is often the more commonly used flavor of automatic differentiation and is usually referred to by the moniker **backpropagation**.

### How reverse mode works

As mentioned, reverse mode proceeds in two steps:

#### The forward pass

The algorithm passes through the computational graph much like forward mode, beginning at the base variables and proceeding through the network until the final output function(s) is (are) reached. Along the way, the value of the function at each node is computed (like in the forward pass). However, the computation of the derivatives is different in that the chain rule is not used. Instead, the derivative of the node is taken only with respect to each parent (i.e. not with respect to each variable). For instance,
$$
    x_5 = x_3 \cdot x_4
$$
Then when we compute the derivative of $x_5$ in the forward pass, we get:
$$
    \dot x_5 = \left(\frac{d}{dx_3}(x_3 \cdot x_4), \frac{d}{dx_4}(x_3\cdot x_4)\right) = (x_4, x_3)
$$

We then evaluate these derivatives at the user-supplied value and continue on through the computational trace.

#### The reverse pass

In the reverse pass, we take all the computed values and derivatives and we use them to compute the derivative of the overall function $f$ with respect to each step in the trace, going from the last node to the variables at the beginning. Say we have a simple function:
$$
    f(x, y, z) = x + y \cdot z
$$
This would map to a simple computational graph:

```
x --> x1 --> x5 (+) --> f
             ^
             |
y --> x2 --> x4 (*)
             ^
z --> x3 ----|
```

We can simply compute the derivative of the function with respect to each node in the computational trace stepping backwards from $x_4$ to $x_1$:

$$
    \bar{x_5} &= \frac{df}{dx_5} = 1 \\
    \bar{x_4} &= \frac{df}{dx_4} = \bar{x_5}\frac{dx_5}{dx_4} \\
    \bar{x_3} &= \frac{df}{dx_3} = \bar{x_4}\frac{dx_4}{dx_3} = \frac{df}{dz} \\
    \bar{x_2} &= \frac{df}{dx_2} = \bar{x_4}\frac{dx_4}{dx_2} = \frac{df}{dy} \\
    \bar{x_1} &= \frac{df}{dx_1} = \bar{x_5}\frac{dx_5}{dx_1} = \frac{df}{dx} \\
$$

When we need to compute, say, $\frac{dx_4}{dx_3}$, we simply sub in the value we computed in the forward pass!

The only complication arises when we have a node in the computational graph with more than one child. For instance:
$$
    f(x, y) = x^2 + x / y
$$
Which corresponds to the following graph:

```
x --> x1 --> x3 (pow)-> x5 (add) --> f
        \               ^
         \              |
y --> x2 --> x4 (div) --+
```

In order to get the derivative of $x_1$, we need to use the chain rule to *add up* the derivative from $x_4$ and $x_3$):
$$
    \bar{x_1} = \frac{df}{dx_1} = \bar{x_4}\frac{dx_4}{dx_1} + \bar{x_3}\frac{dx_3}{dx_1}
$$

Once again, the computation of the full derivative of each node in the reverse pass means that we only need to do _one_ forward and _one_ reverse pass per output function, _regardless_ of the number of input functions.

### Reverse mode implementation

Reverse mode is implemented in `reverse.py`. Two additional classes are created:

1) `ReverseDiff`

The core responsibility of `ReverseDiff` is to wrap an `Expression` object, beginning the forward and reverse passes and collecting the resulting derivatives into a gradient vector or Jacobian matrix.

**Methods and important attributes**

`expr`

The `Expression` object whose derivative is being taken.

`vars`

The correct ordering of the `Var` objects for that `Expression`

`trace`

The nodes in the computational trace. Initially, this is an empty list, but is later populated during a call to `forward`.

`forward(self, expr, *args, child=None)`

Conducts the foward pass. For each `Expression` or `Var` in the overall function to be differentiated, it instantiates a `TraceNode` (more on this below). By the end of the `forward` call, the `trace` attribute of `ReverseDiff` is populated with the full computational trace (each node is represented by a `TraceNode` object).

`reverse(self, var=None)`

Conducts the reverse pass. For each `TraceNode` in the `trace` attribute, the derivative of the overall function is computed with respect to that node. If that `TraceNode` represents one of the base variables (i.e. it contains a `Var` object then that derivative is saved to be included in the gradient or Jacobian.

2) `TraceNode`

The main role of this class is to store the current values and derivatives during the forward pass, as well as the children of that node (since `Expression` objects only contain references to their parents; not their children). This class is also responsible for computing the derivative (`bar`) of the `TraceNode` with respect to the overall function.

**Methods and important attributes**

`expr`

The `Expression` that is represented by this `TraceNode`

`currval`

The current value of this `TraceNode`. Computed during the forward pass.

`derivs`

The derivatives of this `TraceNode` with respect to its two parents (a list of length 1 or 2, depending on if this is a unary or binary expression).

`children`

The `TraceNode`s that represent the children of this `TraceNode`.

`add_child(self, child)`

Adds a reference to another `TraceNode` that represents the child of this `TraceNode`'s `Expression`. This is what allows the reverse pass to go from children to parents.

`bar` (property)

Compute the derivative of the overall function with respect to this `TraceNode`. Or, if the derivative has already been computed (and stored), return the derivative of the overall function with respect to this `TraceNode`.

`deriv_parent(self, parent_expr)`

Compute the derivative of this `TraceNode` with respect to the `Expression` object `parent_expr`. This is called during the reverse pass.

## Future plan

1\. Nth Derivatives

Give user option to specify arbitrary number of times to differentiate function. This can be done by recursively calling our operators to find the nth derivatives.

In the same way, the Jacobian matrix can be extended to the Hessian matrix. Hessian matrix represents 2nd partial derivatives of a vector function.

    
2\. Vector Calculus

Support divergence and curl operators on vector fields.


3.\ Polar and Spherical Coordinates

User can input function in polar or spherical form. Differential operators can return values in the corresponding coordinate system.